# Test the prior model to see how well it works

In [2]:
import numpy as np
import tensorflow as tf

In [112]:
from prior_model import PriorModelBellman

In [113]:
from util import random_observation_sequence, transform_observations
import gym

In [114]:
env = gym.make('MountainCarContinuous-v0')

num_seqs = 1200
seq_length = 12
ob_seqs = []
next_obs = []
rewards = []

for i in range(num_seqs):
    o, a, r = random_observation_sequence(env, seq_length)

    train = o[:-1]
    test = o[-1]

    rewards.append(r)

    ob_seqs.append(train)
    next_obs.append(test)

ob_seqs = np.array(ob_seqs)
next_obs = np.array(next_obs)
rewards = np.array(rewards)
ob_seqs.shape

ob_seqs_stddev = np.ones_like(ob_seqs)
next_obs_stddev = np.ones_like(next_obs)

next_obs
rewards

array([[-1.02089789e-03, -2.63429839e-02, -6.43045760e-02, ...,
        -7.18121325e-02, -1.55811912e-06, -2.54941925e-02],
       [-1.55399723e-02, -8.61167025e-02, -1.01720223e-02, ...,
        -3.65342759e-03, -6.71025815e-02, -2.28344572e-02],
       [-1.56588573e-04, -2.74510544e-03, -2.74038173e-02, ...,
        -3.78842932e-02, -1.30653332e-02, -3.30236791e-02],
       ...,
       [-3.12667365e-02, -1.72479677e-03, -3.02168801e-03, ...,
        -4.32834535e-04, -7.12517285e-02, -1.12898921e-02],
       [-1.08957392e-02, -7.79804716e-02, -5.15022287e-02, ...,
        -5.84160851e-02, -4.15068630e-02, -8.25505603e-02],
       [-4.49697721e-02, -7.81647495e-02, -1.08266080e-02, ...,
        -5.55745053e-02, -1.46954400e-02, -2.66901254e-02]])

In [115]:
ob_seqs[0]

array([[-5.53517997e-01,  0.00000000e+00],
       [-5.53142369e-01,  3.75652395e-04],
       [-5.51775575e-01,  1.36682065e-03],
       [-5.51400542e-01,  3.75043048e-04],
       [-5.51913500e-01, -5.12973871e-04],
       [-5.50954700e-01,  9.58819001e-04],
       [-5.49935400e-01,  1.01928855e-03],
       [-5.48296630e-01,  1.63875264e-03],
       [-5.46126664e-01,  2.16995459e-03],
       [-5.42930782e-01,  3.19589116e-03],
       [-5.40861130e-01,  2.06966931e-03],
       [-5.38667977e-01,  2.19315826e-03]])

In [116]:
np.stack([rewards[0]]*2).T * np.arange(12).reshape(12, 1)

array([[-0.00000000e+00, -0.00000000e+00],
       [-2.63429839e-02, -2.63429839e-02],
       [-1.28609152e-01, -1.28609152e-01],
       [-1.60246455e-01, -1.60246455e-01],
       [-2.82100130e-01, -2.82100130e-01],
       [-4.63810342e-03, -4.63810342e-03],
       [-4.75216788e-02, -4.75216788e-02],
       [-3.72745830e-02, -3.72745830e-02],
       [-2.61202061e-01, -2.61202061e-01],
       [-6.46309192e-01, -6.46309192e-01],
       [-1.55811912e-05, -1.55811912e-05],
       [-2.80436117e-01, -2.80436117e-01]])

In [117]:
pm = PriorModelBellman(2)

In [118]:
pm.train(ob_seqs[0], rewards[0])

1/1 [==============================] - 0s 114ms/step - loss: 4.1720e-04


2022-08-04 19:21:28.791784: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
